# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/jazz.gml")

In [3]:
print(nx.info(g))

Graph with 198 nodes and 2742 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 198 nodes and 2742 edges
Graph with 198 nodes and 2742 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/jazz-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/jazz-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/jazz-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/jazz-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

198

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[21,
 30,
 12,
 15,
 101,
 4,
 9,
 103,
 71,
 93,
 10,
 90,
 41,
 75,
 132,
 73,
 29,
 129,
 100,
 44,
 5,
 6,
 36,
 42,
 3,
 45,
 77,
 85,
 131,
 16,
 91,
 135,
 63,
 18,
 25,
 13,
 1,
 88,
 96,
 43,
 125,
 26,
 106,
 2,
 48,
 159,
 136,
 155,
 47,
 86,
 112,
 149,
 27,
 62,
 37,
 51,
 89,
 80,
 92,
 82,
 67,
 49,
 50,
 56,
 55,
 57,
 99,
 156,
 52,
 54,
 98,
 53,
 28,
 72,
 148,
 152,
 160,
 175,
 118,
 64,
 46,
 108,
 17,
 39,
 114,
 65,
 167,
 107,
 151,
 76,
 61,
 94,
 69,
 74,
 104,
 23,
 95,
 105,
 40,
 97,
 84,
 147,
 180,
 35,
 109,
 87,
 173,
 174,
 0,
 20,
 186,
 70,
 7,
 68,
 66,
 170,
 172,
 171,
 163,
 11,
 79,
 8,
 81,
 22,
 179,
 177,
 130,
 122,
 123,
 31,
 184,
 185,
 113,
 168,
 153,
 169,
 60,
 119,
 178,
 189,
 24,
 176,
 161,
 158,
 117,
 154,
 146,
 157,
 110,
 127,
 58,
 59,
 121,
 142,
 78,
 120,
 128,
 187,
 150,
 124,
 19,
 162,
 115,
 38,
 14,
 111,
 33,
 195,
 102,
 34,
 32,
 116,
 134,
 182,
 183,
 83,
 145,
 140,
 144,
 164,
 137,
 143,
 141,
 191,
 193,


In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{21: 198,
 30: 197,
 12: 196,
 15: 195,
 101: 194,
 4: 193,
 9: 192,
 103: 191,
 71: 190,
 93: 189,
 10: 188,
 90: 187,
 41: 186,
 75: 185,
 132: 184,
 73: 183,
 29: 182,
 129: 181,
 100: 180,
 44: 179,
 5: 178,
 6: 177,
 36: 176,
 42: 175,
 3: 174,
 45: 173,
 77: 172,
 85: 171,
 131: 170,
 16: 169,
 91: 168,
 135: 167,
 63: 166,
 18: 165,
 25: 164,
 13: 163,
 1: 162,
 88: 161,
 96: 160,
 43: 159,
 125: 158,
 26: 157,
 106: 156,
 2: 155,
 48: 154,
 159: 153,
 136: 152,
 155: 151,
 47: 150,
 86: 149,
 112: 148,
 149: 147,
 27: 146,
 62: 145,
 37: 144,
 51: 143,
 89: 142,
 80: 141,
 92: 140,
 82: 139,
 67: 138,
 49: 137,
 50: 136,
 56: 135,
 55: 134,
 57: 133,
 99: 132,
 156: 131,
 52: 130,
 54: 129,
 98: 128,
 53: 127,
 28: 126,
 72: 125,
 148: 124,
 152: 123,
 160: 122,
 175: 121,
 118: 120,
 64: 119,
 46: 118,
 108: 117,
 17: 116,
 39: 115,
 114: 114,
 65: 113,
 167: 112,
 107: 111,
 151: 110,
 76: 109,
 61: 108,
 94: 107,
 69: 106,
 74: 105,
 104: 104,
 23: 103,
 95: 102,
 105: 101,


In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Misc-Jazz/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Misc-Jazz/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True